# Uncertainty in Estimation
In the previous section, we took a detour into the philosophy of science and the logical ground upon which statistical inference is based. For the moment, let us put that discussion to one side and get back to our statistical model. We will see where the problem of induction comes into play when we start discussing null hypothesis significance testing later in the lesson.

## Parameter Estimates as Random Variables